In [198]:
import pandas as pd
import polars as pl
import numpy as np

from typing import Optional

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors

from catboost import CatBoostRegressor, Pool

import warnings
warnings.filterwarnings("ignore", message='not removing hydrogen atom with dummy atom neighbors')


def read(n: str, **kwargs):
    return pl.read_csv(f"../data/{n}", **kwargs).to_pandas()


data = read("data.csv")

In [262]:
# 0.3
# RMSE: 2.147, SPLIT_SEED: 27818, MODEL_SEED: 13700
# RMSE: 2.107, SEED: 48362
# RMSE: 2.789, SPLIT_SEED: 51225, MODEL_SEED: 78855
# RMSE: 2.296, SPLIT_SEED: 43773, MODEL_SEED: 55376
# 0.2
# RMSE: 1.918, SPLIT_SEED: 24195, MODEL_SEED: 27886


from sklearn.ensemble import ExtraTreesRegressor

scaler = StandardScaler()
selector = SelectKBest(score_func=f_regression, k=100)

split_seed = np.random.randint(0, 100000)
model_seed = np.random.randint(0, 100000)

def preprocess(
    data: pd.DataFrame,
) -> tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    data = data.copy()
    data = data.dropna()
    data = data.drop_duplicates()

    def compute_descriptors(smiles: str):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        features = {
            "MolLogP": Descriptors.MolLogP(mol),
            "NumRotatableBonds": rdMolDescriptors.CalcNumRotatableBonds(mol),
            "NumAromaticRings": Descriptors.NumAromaticRings(mol),
            "NumSaturatedRings": Descriptors.NumSaturatedRings(mol),
            "NumAtoms": mol.GetNumAtoms(),
            "NumHeavyAtoms": mol.GetNumHeavyAtoms(),
            "MolWt": Descriptors.MolWt(mol),
            "TPSA": Descriptors.TPSA(mol),
            "NumHAcceptors": Descriptors.NumHAcceptors(mol),
        }

        morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        morgan_fp_bits = np.array(list(morgan_fp.ToBitString())).astype(int)
        for i, bit in enumerate(morgan_fp_bits):
            features[f"Bit_{i}"] = bit

        return features

    descriptors_list = (
        data["smiles"].apply(compute_descriptors).dropna().tolist()
    )
    descriptors_df = pd.DataFrame.from_records(descriptors_list)

    descriptors_df = descriptors_df.reset_index(drop=True)
    data = data.reset_index(drop=True)

    data = pd.concat([descriptors_df, data], axis=1)
    data = data.drop("smiles", axis=1)

    if "id" in data.columns:
        data = data.drop("id", axis=1)

    numeric_cols = data.select_dtypes(include=[np.number]).columns
    imputer = SimpleImputer(strategy="mean")
    data_imputed = imputer.fit_transform(data[numeric_cols])

    data = pd.DataFrame(data_imputed, columns=numeric_cols)

    if "logK" in data.columns:
        y = data.pop("logK")
        data = scaler.fit_transform(data, y)
        data = selector.fit_transform(data, y)
        return data, y

    data = scaler.transform(data)
    data = selector.transform(data)
    return data, None


X, y = preprocess(data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=split_seed)

# X_train, y_train = preprocess(train)

# check = test.copy()
# y_test = test.pop("logK")
# X_test, _ = preprocess(check)

# train_pool = Pool(X_train, y_train)
# test_pool = Pool(X_test, y_test)

model = ExtraTreesRegressor(random_state=model_seed)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse:.3f}, SPLIT_SEED: {split_seed}, MODEL_SEED: {model_seed}")

RMSE: 2.919, SPLIT_SEED: 33161, MODEL_SEED: 43047


In [247]:
import joblib

joblib.dump(model, "../models/30pS_ETR_2.296_logk.joblib");

In [200]:
model = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    task_type="GPU",
    devices="0",
    verbose=False,
)


model.fit(train_pool, verbose=100, eval_set=test_pool, early_stopping_rounds=20)

predictions = model.predict(test_pool)

mse = mean_squared_error(y_test, predictions)
rmse = mse**0.5
print(f"Root Mean Squared Error (RMSE): {rmse}")

Learning rate set to 0.052109
0:	learn: 6.1475510	test: 5.0978660	best: 5.0978660 (0)	total: 15.5ms	remaining: 15.5s
bestTest = 3.000267424
bestIteration = 55
Shrink model to first 56 iterations.
Root Mean Squared Error (RMSE): 3.0002667463224255


In [201]:
model.save_model("../models/best_model.cbm")

In [213]:
from sklearn.ensemble import ExtraTreesRegressor

model = ExtraTreesRegressor(random_state=seed)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse:.3f}, SEED: {seed}")
# RMSE: 1.788, SEED: 8779
# RMSE: 1.731, SEED: 61821
# RMSE: 1.723, SEED: 61311


RMSE: 2.462, SEED: 1635


In [157]:
import joblib

joblib.dump(model, "../models/ETR_logK.joblib");

In [188]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import GammaRegressor
import numpy as np

# Определение модели
model = GammaRegressor()

# Сетка гиперпараметров для тестирования
param_distributions = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],  # Регуляризация
    'max_iter': [100, 200, 300, 500],  # Максимальное количество итераций
    'tol': [1e-3, 1e-4, 1e-5],  # Точность решения
}

# Настройка RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Обучение RandomizedSearchCV
random_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Best params:", random_search.best_params_)

# Использование лучшей модели для предсказаний
best_model = random_search.best_estimator_
predictions = best_model.predict(X_test)

# Расчет RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse:.3f}")


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best params: {'tol': 0.001, 'max_iter': 100, 'alpha': 1.0}
RMSE: 2.786


In [11]:
rmse_history = []

In [177]:
from lazypredict.Supervised import LazyRegressor

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=rmse)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
rmse_history.append(models["RMSE"])

models_df = models.sort_values("RMSE")
if len(rmse_history) > 1:
    models_df["RMSE_diff"] = rmse_history[-1] - rmse_history[-2]
else:
    models_df["RMSE_diff"] = np.nan

models_df[["RMSE", "RMSE_diff"]]

100%|██████████| 42/42 [00:02<00:00, 18.11it/s]


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 172, number of used features: 27
[LightGBM] [Info] Start training from score 9.227965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

,RMSE,RMSE_diff
Model,,
LassoCV,2.78,-0.13
GammaRegressor,2.78,-0.20
ElasticNetCV,2.79,0.04
TweedieRegressor,2.80,0.13
BayesianRidge,2.82,0.10
RidgeCV,2.86,-0.13
ExtraTreesRegressor,2.87,1.04
SGDRegressor,2.89,-0.41
KNeighborsRegressor,2.91,0.10


In [ ]:
from rdkit.Chem import rdMolDescriptors, GetSSSR

smiles = "O=C(O)[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO"
mol = Chem.MolFromSmiles(smiles)

len(GetSSSR(mol)), mol.GetNumHeavyAtoms(), mol.GetNumAtoms(), rdMolDescriptors.CalcNumRotatableBonds(mol)